In [1]:
!pip install -q transformers accelerate gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 17.8 MB/s eta 0:00:00


In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load tokenizer and model with chat formatting support
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto"
)

# Simple pipeline (optional)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


Device set to use cpu


In [8]:
# Custom mythic prompt template
def generate_myth(seed, mode, word_count):
    style = "poetic" if mode.lower() == "poem" else "mythic and narrative"

    prompt = f"""
You are an ancient storyteller, a weaver of timeless legends.

Create a {mode.lower()} with a clear beginning, middle, and end, inspired by the myth seed: "{seed}".
Use vivid imagery, rich language, and mythic proportions. Describe an unreal world, its eerie landscapes, forgotten gods, and strange rituals.
Characters should feel ancient, complex, and born from the bones of folklore.

Begin now, in a tone that echoes through time:
"""
    max_tokens = int(word_count * 1.5)

    generated = pipe(
        prompt,
        max_new_tokens=max_tokens,
        temperature=0.95,
        top_p=0.92,
        repetition_penalty=1.2,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
    )[0]["generated_text"]

    # Only return the part after our prompt
    return generated[len(prompt):].strip()


In [9]:
import gradio as gr

# Gradio UI
with gr.Blocks() as ui:
    gr.Markdown("## 🪷 MythMaker Lite: Mythic Generator\nCraft vivid legends from a single seed 🌑")

    with gr.Row():
        seed_input = gr.Textbox(label="🔮 Myth Seed", placeholder="e.g., Bloody Mist Village")
        mode_choice = gr.Radio(["Poem", "Story"], label="📜 Select Type", value="Story")
        word_slider = gr.Slider(minimum=50, maximum=300, value=150, step=10, label="🔠 Approx. Word Count")

    gen_btn = gr.Button("🌀 Weave the Myth")
    output_box = gr.Textbox(label="🌌 Generated Tale", lines=20)

    gen_btn.click(fn=generate_myth, inputs=[seed_input, mode_choice, word_slider], outputs=output_box)

ui.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://694f5fbe7ff8f00049.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
